In [49]:
import pandas as pd
import pyarrow.parquet as pq
from plotnine import *
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt
from lifelines.statistics import logrank_test
import numpy as np
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.util import Surv
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import make_scorer
from sksurv.metrics import concordance_index_censored
from statsmodels.stats.outliers_influence import variance_inflation_factor
from lifelines import CoxPHFitter
from sklearn.metrics import mean_squared_error
from lifelines import WeibullAFTFitter
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import brier_score
from scipy.integrate import trapezoid
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay, accuracy_score,
    precision_score, recall_score, roc_auc_score
)
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs

# Import Data into df

In [ ]:
file_path = ""
# Read the Parquet file
table = pq.read_table(file_path)

# Convert to a pandas DataFrame if needed
df = table.to_pandas()

# Display the first few rows of the DataFrame
df.head()

# Contract Renewal Features

In [51]:
# Ensure 'date' is in datetime format
df["calendar_date"] = pd.to_datetime(df["calendar_date"])
df["target_date"] = pd.to_datetime(df["target_date"])
df["contract_end_date"] = pd.to_datetime(df["contract_end_date"])

## remain_contract_month

In [52]:
# remain_contract_month: months left in a contract and ensure it doesn't go below 0
df['remain_contract_month'] = ((df['contract_end_date'] - df['calendar_date']).dt.days / 30).round()
df['remain_contract_month'] = df['remain_contract_month'].clip(lower=0)

## df_changes

extracting date information and grouping by contract_end_date per customer. 

df_changes contains repeats of customer id's but only one contract per row!

In [53]:
# Convert the column to datetime format
df['contract_end_date'] = pd.to_datetime(df['contract_end_date'])

# Extract year, month, and day as separate columns
df['year'] = df['contract_end_date'].dt.year
df['month'] = df['contract_end_date'].dt.month
df['day'] = df['contract_end_date'].dt.day

# Identify rows where any of year, month, or day changes compared to the previous row
df_changes = df[(df['year'].ne(df['year'].shift())) |
                (df['month'].ne(df['month'].shift())) |
                (df['day'].ne(df['day'].shift())) |
                (df['time_step'] == 1)] 

df_changes = df_changes[df_changes['remain_contract_month'] > 0]

In [ ]:
# Display the resulting DataFrame with changes
df_changes

In [ ]:
df_changes[(df_changes['time_step']==1)&(df_changes['remain_contract_month']==97)]

In [ ]:
df_changes[df_changes['time_step']==1]['remain_contract_month'].describe()

In [ ]:
df_changes[(df_changes['cat1']=='A') & (df_changes['cat2']=='A') & (df_changes['cat3']=='B')& (df_changes['time_step']==1)]['remain_contract_month'].describe()

### calculating total contract count for each customer

In [ ]:
total_contract_counts = df_changes.groupby('id').size().reset_index(name='total_contract_count')
total_contract_counts.head()

### contract length calculations

In [ ]:
# Extract the last contract length for each customer
last_contracts = df_changes.groupby('id').last().reset_index()

# Compute contract length statistics, including last contract length
contract_lengths = df_changes.groupby('id')['remain_contract_month'].agg(
    avg_contract_length='mean',
    longest_contract='max',
    shortest_contract='min',
    std_contract_length='std'
).reset_index()

# Replace NaN in standard deviation with 0
contract_lengths['std_contract_length'] = contract_lengths['std_contract_length'].fillna(0)

# Merge the last contract length into contract_lengths
contract_lengths = contract_lengths.merge(
    last_contracts[['id', 'remain_contract_month']], 
    on='id', 
    how='left'
).rename(columns={'remain_contract_month': 'last_contract_length'})

# last_contract_length / avg_contract_length
contract_lengths['last_avg_contract_ratio'] = contract_lengths['last_contract_length'] / contract_lengths['avg_contract_length']

# last_contract_length / longest_contract
# contract_lengths['last_peak_contract_ratio'] = contract_lengths['last_contract_length'] / contract_lengths['longest_contract']

# Display the new DataFrame
contract_lengths.head()

If the ratio is close to 1:
- The customer's last contract length is consistent with their historical contract lengths, suggesting a stable renewal pattern.
- This could indicate that the customer is not changing their behavior significantly before churning (or they may not churn at all).
- **if its 1 that means that ALL their contracts are the same length (probable that they only had 1 contract)**

If the ratio is lower than 1:
- The last contract is shorter than their average contract length.
- This might indicate that the customer is hesitating to commit, testing shorter-term contracts before churning.
- **they may be shortening their contracts to leave BUT need to establish what ratio is significantly low enough**

If the ratio is greater than 1:
- The last contract is longer than their historical average.
- This could mean the customer has decided to stay longer than usual, perhaps due to a better offer or a change in needs.
- **they may be interested in strengthening their commitments with F5**

## Base data preparation

In [60]:
#churned customer at the last month before target date
churn_last = df[(df['target_label'] == 1) & (df['calendar_date'] == df['target_date']- pd.Timedelta(days=1))]

In [61]:
churn = df[(df['target_label'] == 1) & (df['time_step'] == 1)]
churn = churn[churn['contract_end_date']>churn['calendar_date']]

#churned customer at the first time step of calendar date
churn = churn[['id','calendar_date','target_label','target_date']]
#combined churned customer's the first time step of calendar date with the data at the last month before target date
churn_info = churn_last[['id','time_step','contract_end_date','x1','x2','x1_cumulative','x2_cumulative','cat1','cat2','cat3']]
churn = pd.merge(churn, churn_info, on='id', how='inner')
churn = churn.rename(columns={'time_step':'life'})
churn = churn[(churn['x1'] != 0) & (churn['x2'] != 0)]
churn = churn[churn['contract_end_date']<churn['target_date']]
churn = churn.rename(columns={'calendar_date': 'join_date'})
churn = churn.rename(columns={'x1': 'last_x1'})
churn = churn.rename(columns={'x2': 'last_x2'})
churn['x1/x2'] = churn['x1_cumulative'] / churn['x2_cumulative']
churn['x1_renew_time'] = churn['x1_cumulative'] / churn['last_x1']
churn['x2_renew_time'] = churn['x2_cumulative'] / churn['last_x2']

In [62]:
active_join = df[(df['target_label']==0) & (df['time_step']==1)]
active_join = active_join[['id','calendar_date']]
active_join = active_join.rename(columns={'calendar_date':'join_date'})

In [63]:
active = df[(df['target_label']==0) & (df['calendar_date']=='2024-09-30') & (df['x1'] != 0) & (df['x2'] != 0)]
active = active[(active['customer_status']!='inactive')]
active['life'] = active['time_step']
active = active.drop(columns=['calendar_date','time_step','customer_status'])
active = active.rename(columns={'x1': 'last_x1'})
active = active.rename(columns={'x2': 'last_x2'})
active['x1/x2'] = active['x1_cumulative'] / active['x2_cumulative']
active['x1_renew_time'] = active['x1_cumulative'] / active['last_x1']
active['x2_renew_time'] = active['x2_cumulative'] / active['last_x2']
active = pd.merge(active_join, active, on='id', how='inner')
active = active[active['contract_end_date']<'2040-01-31']

# Creating 'clean' dataset

In [64]:
clean = pd.concat([active,churn])

# Creating agg_stats for x1 & x2

In [ ]:
# Filter the DataFrame for rows where calendar_date is prior to target_date or target_label == 0
filtered_df = df[(df['calendar_date'] < df['target_date']) | (df['target_label'] == 0)]

# Define the columns for which you want to calculate statistics
columns_to_aggregate = ['x1', 'x2', 'x1_cumulative', 'x2_cumulative']

# Group by 'id' and calculate the mean, median, min, max, and std for the selected columns
agg_stats = filtered_df.groupby('id')[columns_to_aggregate].agg(['mean', 'median', 'min', 'max', 'std'])

# Flatten the multi-level column index
agg_stats.columns = ['_'.join(col) for col in agg_stats.columns]

# Reset the index to convert the result back into a DataFrame
agg_stats = agg_stats.reset_index()

# Display the aggregated statistics
print(agg_stats)

agg_stats.describe()


# Merging datasets INTO clean

agg_stats, contract_lengths, total_contract_counts

In [66]:
clean = clean.merge(agg_stats, on='id', how='left')
clean = clean.merge(contract_lengths, on='id', how='left')
clean = clean.merge(total_contract_counts, on='id', how='left')

In [67]:
clean['x1_std'] = clean['x1_std'].fillna(0)
clean['x2_std'] = clean['x2_std'].fillna(0)
clean['x1_cumulative_std'] = clean['x1_cumulative_std'].fillna(0)
clean['x2_cumulative_std'] = clean['x2_cumulative_std'].fillna(0)

# Create dataset at certain time point

In [68]:
def create_df(df, time_point, future_point, id_list):
    """
    Process the dataframe based on specified conditions and return a new dataframe.
    
    Args:
        df (pd.DataFrame): The input dataframe.
        time_point (int/float): The reference time point.
        future_point (int/float): The future time point to evaluate customer status.
        id_list (list): List of IDs to filter.
    
    Returns:
        pd.DataFrame: Processed dataframe with aggregated statistics.
    """
    # Filter dataframe based on id_list
    if isinstance(id_list, pd.DataFrame):
        id_list = id_list['id'].tolist()  # Convert column to a list
    filtered_df = df[df['id'].isin(id_list)]

    # Exclude IDs where customer_status is 'inactive' at time_step == time_point
    active_at_time_point = filtered_df[(filtered_df['time_step'] == time_point) & (filtered_df['customer_status'] != 'inactive')]['id'].unique()
    filtered_df = filtered_df[filtered_df['id'].isin(active_at_time_point)]

    # Create target_label based on future customer status
    future_status = filtered_df[filtered_df['time_step'] == future_point]
    valid_future_ids = future_status['id'].unique()
    
    # Remove IDs that do not have future_point data
    filtered_df = filtered_df[filtered_df['id'].isin(valid_future_ids)]
    
    target_map = dict(zip(future_status['id'], (future_status['customer_status'] == 'inactive').astype(int)))
    filtered_df['target_label'] = filtered_df['id'].map(target_map).fillna(0).astype(int)
    
    # Filter rows where time_step <= time_point
    filtered_df = filtered_df[filtered_df['time_step'] <= time_point]
    
    # Define columns for aggregation
    columns_to_aggregate = ['x1', 'x2', 'x1_cumulative', 'x2_cumulative']
    
    # Group by 'id' and compute statistical aggregations
    agg_stats = filtered_df.groupby('id')[columns_to_aggregate].agg(['mean', 'median', 'min', 'max', 'std'])
    
    # Flatten multi-level columns
    agg_stats.columns = ['_'.join(col) for col in agg_stats.columns]
    
    # Reset index
    agg_stats = agg_stats.reset_index()

    filtered_df['remain_contract_month'] = ((filtered_df['contract_end_date'] - filtered_df['calendar_date']).dt.days / 30).round()
    filtered_df['remain_contract_month'] = filtered_df['remain_contract_month'].clip(lower=0)

    # Identify rows where any of year, month, or day changes compared to the previous row
    df_changes = filtered_df[(filtered_df['year'].ne(filtered_df['year'].shift())) |
                (filtered_df['month'].ne(filtered_df['month'].shift())) |
                (filtered_df['day'].ne(filtered_df['day'].shift())) |
                (filtered_df['time_step'] == 1)] 
    df_changes = df_changes[df_changes['remain_contract_month'] > 0]

    # Extract the last contract length for each customer
    last_contracts = df_changes.groupby('id').last().reset_index()

    # Compute contract length statistics, replacing NaN std with 0
    contract_lengths = df_changes.groupby('id')['remain_contract_month'].agg(
    avg_contract_length='mean',
    longest_contract='max',
    shortest_contract='min',
    std_contract_length='std'
    ).reset_index()

    # Replace NaN in standard deviation with 0
    contract_lengths['std_contract_length'] = contract_lengths['std_contract_length'].fillna(0)

    # Merge the last contract length into contract_lengths
    contract_lengths = contract_lengths.merge(
        last_contracts[['id', 'remain_contract_month']], 
        on='id', 
        how='left'
    ).rename(columns={'remain_contract_month': 'last_contract_length'})

    # last_contract_length / avg_contract_length
    contract_lengths['last_avg_contract_ratio'] = contract_lengths['last_contract_length'] / contract_lengths['avg_contract_length']


    total_contract_counts = df_changes.groupby('id').size().reset_index(name='total_contract_count')


    # Take only the rows where `time_step == time_point` for feature engineering
    filtered_df = filtered_df[filtered_df['time_step'] == time_point]
    
    # Rename columns
    filtered_df = filtered_df.rename(columns={'x1': 'last_x1', 'x2': 'last_x2'})

    # Rename columns
    filtered_df = filtered_df.rename(columns={'x1': 'last_x1', 'x2': 'last_x2'})
    
    # Compute derived features
    filtered_df['x1/x2'] = filtered_df['x1_cumulative'] / filtered_df['x2_cumulative']
    filtered_df['x1_renew_time'] = filtered_df['x1_cumulative'] / filtered_df['last_x1']
    filtered_df['x2_renew_time'] = filtered_df['x2_cumulative'] / filtered_df['last_x2']

    # Merge with agg_stats
    filtered_df = filtered_df.merge(agg_stats, on='id', how='left')
    filtered_df = filtered_df.merge(contract_lengths, on='id', how='left')
    filtered_df = filtered_df.merge(total_contract_counts, on='id', how='left')


    return filtered_df


# Clustering

## Create datasets for clustering (only cat1 and cat2)

In [69]:
clean['Category_Combo2'] = clean['cat1'] + '-' + clean['cat2']

In [70]:
cluster_id = clean[~clean['Category_Combo2'].isin(['N/A-A', 'D-N/A', 'C-N/A'])]['id'].to_list()

In [71]:
cluster_12 = create_df(df, 12, 12, cluster_id)
cluster_12 = cluster_12.merge(clean[['id', 'Category_Combo2']], on='id', how='left')

cluster_24 = create_df(df, 24, 24, cluster_id)
cluster_24 = cluster_24.merge(clean[['id', 'Category_Combo2']], on='id', how='left')

cluster_36 = create_df(df, 36, 36, cluster_id)
cluster_36 = cluster_36.merge(clean[['id', 'Category_Combo2']], on='id', how='left') 


In [72]:
cluster_18 = create_df(df, 18, 18, cluster_id)
cluster_18 = cluster_18.merge(clean[['id', 'Category_Combo2']], on='id', how='left') 

cluster_30 = create_df(df, 30, 30, cluster_id)
cluster_30 = cluster_30.merge(clean[['id', 'Category_Combo2']], on='id', how='left') 

cluster_48 = create_df(df, 48, 48, cluster_id)
cluster_48 = cluster_48.merge(clean[['id', 'Category_Combo2']], on='id', how='left') 

In [73]:
cluster_12_median = cluster_12.groupby('Category_Combo2')[['last_x1', 'last_x2', 'x1_cumulative', 'x2_cumulative','x1/x2', 'x1_renew_time', 'x2_renew_time', 'x1_mean', 'x1_median',
       'x1_min', 'x1_max', 'x1_std', 'x2_mean', 'x2_median', 'x2_min',
       'x2_max', 'x2_std', 'x1_cumulative_mean', 'x1_cumulative_median',
       'x1_cumulative_min', 'x1_cumulative_max', 'x1_cumulative_std',
       'x2_cumulative_mean', 'x2_cumulative_median', 'x2_cumulative_min',
       'x2_cumulative_max', 'x2_cumulative_std', 'avg_contract_length',
       'longest_contract', 'shortest_contract', 'std_contract_length',
       'last_contract_length', 'last_avg_contract_ratio',
       'total_contract_count']].median().reset_index()

cluster_24_median = cluster_24.groupby('Category_Combo2')[['last_x1', 'last_x2', 'x1_cumulative', 'x2_cumulative','x1/x2', 'x1_renew_time', 'x2_renew_time', 'x1_mean', 'x1_median',
       'x1_min', 'x1_max', 'x1_std', 'x2_mean', 'x2_median', 'x2_min',
       'x2_max', 'x2_std', 'x1_cumulative_mean', 'x1_cumulative_median',
       'x1_cumulative_min', 'x1_cumulative_max', 'x1_cumulative_std',
       'x2_cumulative_mean', 'x2_cumulative_median', 'x2_cumulative_min',
       'x2_cumulative_max', 'x2_cumulative_std', 'avg_contract_length',
       'longest_contract', 'shortest_contract', 'std_contract_length',
       'last_contract_length', 'last_avg_contract_ratio',
       'total_contract_count']].median().reset_index()

cluster_36_median = cluster_36.groupby('Category_Combo2')[['last_x1', 'last_x2', 'x1_cumulative', 'x2_cumulative','x1/x2', 'x1_renew_time', 'x2_renew_time', 'x1_mean', 'x1_median',
       'x1_min', 'x1_max', 'x1_std', 'x2_mean', 'x2_median', 'x2_min',
       'x2_max', 'x2_std', 'x1_cumulative_mean', 'x1_cumulative_median',
       'x1_cumulative_min', 'x1_cumulative_max', 'x1_cumulative_std',
       'x2_cumulative_mean', 'x2_cumulative_median', 'x2_cumulative_min',
       'x2_cumulative_max', 'x2_cumulative_std', 'avg_contract_length',
       'longest_contract', 'shortest_contract', 'std_contract_length',
       'last_contract_length', 'last_avg_contract_ratio',
       'total_contract_count']].median().reset_index()


In [74]:
cluster_18_median = cluster_18.groupby('Category_Combo2')[['last_x1', 'last_x2', 'x1_cumulative', 'x2_cumulative','x1/x2', 'x1_renew_time', 'x2_renew_time', 'x1_mean', 'x1_median',
       'x1_min', 'x1_max', 'x1_std', 'x2_mean', 'x2_median', 'x2_min',
       'x2_max', 'x2_std', 'x1_cumulative_mean', 'x1_cumulative_median',
       'x1_cumulative_min', 'x1_cumulative_max', 'x1_cumulative_std',
       'x2_cumulative_mean', 'x2_cumulative_median', 'x2_cumulative_min',
       'x2_cumulative_max', 'x2_cumulative_std', 'avg_contract_length',
       'longest_contract', 'shortest_contract', 'std_contract_length',
       'last_contract_length', 'last_avg_contract_ratio',
       'total_contract_count']].median().reset_index()

cluster_30_median = cluster_30.groupby('Category_Combo2')[['last_x1', 'last_x2', 'x1_cumulative', 'x2_cumulative','x1/x2', 'x1_renew_time', 'x2_renew_time', 'x1_mean', 'x1_median',
       'x1_min', 'x1_max', 'x1_std', 'x2_mean', 'x2_median', 'x2_min',
       'x2_max', 'x2_std', 'x1_cumulative_mean', 'x1_cumulative_median',
       'x1_cumulative_min', 'x1_cumulative_max', 'x1_cumulative_std',
       'x2_cumulative_mean', 'x2_cumulative_median', 'x2_cumulative_min',
       'x2_cumulative_max', 'x2_cumulative_std', 'avg_contract_length',
       'longest_contract', 'shortest_contract', 'std_contract_length',
       'last_contract_length', 'last_avg_contract_ratio',
       'total_contract_count']].median().reset_index()

cluster_48_median = cluster_48.groupby('Category_Combo2')[['last_x1', 'last_x2', 'x1_cumulative', 'x2_cumulative','x1/x2', 'x1_renew_time', 'x2_renew_time', 'x1_mean', 'x1_median',
       'x1_min', 'x1_max', 'x1_std', 'x2_mean', 'x2_median', 'x2_min',
       'x2_max', 'x2_std', 'x1_cumulative_mean', 'x1_cumulative_median',
       'x1_cumulative_min', 'x1_cumulative_max', 'x1_cumulative_std',
       'x2_cumulative_mean', 'x2_cumulative_median', 'x2_cumulative_min',
       'x2_cumulative_max', 'x2_cumulative_std', 'avg_contract_length',
       'longest_contract', 'shortest_contract', 'std_contract_length',
       'last_contract_length', 'last_avg_contract_ratio',
       'total_contract_count']].median().reset_index()


In [ ]:
# mean of the median
cluster_median_all = pd.concat([cluster_12_median, cluster_24_median, cluster_36_median,cluster_18_median, cluster_30_median, cluster_48_median], ignore_index=True)
cluster_average_all = cluster_median_all.groupby('Category_Combo2')[['last_x1', 'last_x2', 'x1_cumulative', 'x2_cumulative','x1/x2',
        'x1_renew_time', 'x2_renew_time', 'x1_mean', 'x1_median',
       'x1_min', 'x1_max', 'x1_std', 'x2_mean', 'x2_median', 'x2_min',
       'x2_max', 'x2_std', 'x1_cumulative_mean', 'x1_cumulative_median',
       'x1_cumulative_min', 'x1_cumulative_max', 'x1_cumulative_std',
       'x2_cumulative_mean', 'x2_cumulative_median', 'x2_cumulative_min',
       'x2_cumulative_max', 'x2_cumulative_std', 'avg_contract_length',
       'longest_contract', 'shortest_contract', 'std_contract_length',
       'last_contract_length', 'last_avg_contract_ratio',
       'total_contract_count']].mean().reset_index()

cluster_average_all.head()

## PCA (only cat1 and cat2)

In [76]:
cluster_feature = cluster_average_all[[
    'last_x1', 'x1/x2', 'x2_cumulative_min', 
    'avg_contract_length', 'longest_contract', 'total_contract_count']]

In [77]:
# Generate synthetic data
np.random.seed(42)

# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(cluster_feature)

# Apply PCA
pca = PCA(n_components=4)  # Reduce to 4 components
data_pca = pca.fit_transform(data_scaled)

## Pick K

In [ ]:
# Range of clusters to try
k_values = range(1, 11)
wcss = []  # List to store Within-Cluster Sum of Squares

# Calculate WCSS for each value of k
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_pca)
    wcss.append(kmeans.inertia_)  # Inertia is the WCSS

# Plot the Elbow Method graph
plt.figure(figsize=(8, 5))
plt.plot(k_values, wcss, marker='o', linestyle='-', color='blue')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.xticks(k_values)
plt.show()

## Kmeans

In [ ]:


# Apply K-Means Clustering
kmeans = KMeans(n_clusters=7, random_state=42)
clusters = kmeans.fit_predict(data_pca)

cluster_colors = {0: 'blue', 1: 'green', 2: 'orange', 3: 'purple', 4: 'red', 5: 'black', 6: 'grey'}

plt.figure(figsize=(10,8))
for cluster, color in cluster_colors.items():
    subset = data_pca[clusters == cluster]
    plt.scatter(subset[:, 0], subset[:, 1], color=color, label=f'Cluster {cluster}', alpha=0.6)

plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.title('Clusters after PCA')
plt.legend()
plt.show()

In [ ]:
# Creating a DataFrame to inspect the principal components resulting from PCA transformation

pd.DataFrame(pca.components_, columns=cluster_feature.columns, index=[f'PC{i+1}' for i in range(pca.n_components_)])

In [ ]:
# Getting the cluster centers from the KMeans model after clustering in the PCA-transformed space
cluster_centers_pca = kmeans.cluster_centers_
pd.DataFrame(pca.inverse_transform(cluster_centers_pca), columns=cluster_feature.columns)

In [82]:
cluster_average_all['cluster'] = clusters

In [ ]:
cluster_average_all['Final_Profile'] = cluster_average_all['cluster'].replace({
    0: "Big Fish",
    1: "Hidden Gem",
    2: "Rising Legacy",
    3: "Fairweather Friend",
    4: "Small Fish",
    5: "Legacy",
    6: "Silent Volcano"   
})

profile_breakdown = cluster_average_all[['Category_Combo2', 'Final_Profile']].drop_duplicates().groupby('Final_Profile')['Category_Combo2'].apply(list)
for profile, categories in profile_breakdown.items():
    print(f"\n📌 {profile} ({len(categories)} groups):")
    print(categories)

In [84]:
cluster_plot = cluster_average_all[[
    'Category_Combo2','cluster','last_x1', 'x1/x2', 'x2_cumulative_min', 
    'avg_contract_length', 'longest_contract', 'total_contract_count']]

In [ ]:
cluster_plot.groupby('cluster')[['last_x1', 'x1/x2', 'x2_cumulative_min', 
    'avg_contract_length', 'longest_contract', 'total_contract_count']].agg('mean')

## Spectral Cluster

In [ ]:
# Ensure the data is in array format
#X = data_pca.to_numpy()

# Spectral Clustering
sc = SpectralClustering(n_clusters=7, affinity='nearest_neighbors', assign_labels='kmeans', random_state=42)
clusters2 = sc.fit_predict(data_scaled)

# Plot using meaningful features (or first two PCA components)
plt.figure(figsize=(8, 6))
plt.scatter(data_scaled[:, 0], data_scaled[:, 1], c=clusters2, cmap='viridis', s=50, alpha=0.7)
plt.xlabel('last_x1')   # Update to match actual feature names
plt.ylabel('avg_contract_length')
plt.title('Spectral Clustering on Original Data')
plt.show()


In [87]:
cluster_average_all['cluster2'] = clusters2

In [ ]:
cluster_average_all.groupby('cluster2')[['last_x1', 'x1/x2', 'x2_cumulative_min', 
    'avg_contract_length', 'longest_contract', 'total_contract_count']].agg('mean')

In [89]:
silhouette_kmeans = silhouette_score(data_pca, clusters)
silhouette_spectral = silhouette_score(data_scaled, clusters2)

In [ ]:
print(f"KMeans Silhouette: {silhouette_kmeans:.4f}")
print(f"Spectral Silhouette: {silhouette_spectral:.4f}")
# KMeans has lower Silhouette score, we decide to use KMeans instead of Spectral 

# Kaplan-Meier Estimator

In [ ]:
clean.head()

In [ ]:
# Prepare data
T = clean['life']  # Time to event (e.g., duration in days)
E = clean['target_label']  # Event indicator (1=event occurred, 0=censored)

# Fit Kaplan-Meier estimator
kmf = KaplanMeierFitter()
kmf.fit(T, event_observed=E)

# Plot survival curve
kmf.plot_survival_function()
plt.title('Kaplan-Meier Survival Curve')
plt.xlabel('Time(month)')
plt.ylabel('Survival Probability')
plt.show()

# Cox Proportional Hazards Model
Once a Cox model is fitted, you can generate survival curves based on specific predictor values.

'last_x1': [20], 'last_x2': [20],  
'x1_cumulative': [50], 'x2_cumulative': [50]

In [ ]:
# Fit Cox Proportional Hazards model
cox_df = clean[['life', 'target_label', 'last_x1', 'last_x2', 'x1_cumulative', 'x2_cumulative']]
cox = CoxPHFitter()
cox.fit(cox_df, duration_col='life', event_col='target_label')

# Predict survival function for a specific individual
individual = pd.DataFrame({
    'last_x1': [20],
    'last_x2': [20],
    'x1_cumulative': [50],
    'x2_cumulative': [50]
})
survival_curve = cox.predict_survival_function(individual)

# Plot survival curve
survival_curve.plot()
plt.title('Survival Curve for Individual')
plt.xlabel('Time')
plt.ylabel('Survival Probability')
plt.show()

'last_x1': [2], 'last_x2': [2],  
'x1_cumulative': [5], 'x2_cumulative': [5]

In [ ]:
individual = pd.DataFrame({
    'last_x1': [2],
    'last_x2': [2],
    'x1_cumulative': [5],
    'x2_cumulative': [5]
})
survival_curve = cox.predict_survival_function(individual)

# Plot survival curve
survival_curve.plot()
plt.title('Survival Curve for Individual')
plt.xlabel('Time')
plt.ylabel('Survival Probability')
plt.show()

# Comparing Groups
To compare survival curves for groups

In [ ]:
for group in clean['cat1'].unique():
    group_data = clean[clean['cat1'] == group]
    kmf.fit(group_data['life'], event_observed=group_data['target_label'], label=str(group))
    kmf.plot_survival_function()

plt.title('Survival Curves by Group(cat1)')
plt.xlabel('Time(month)')
plt.ylabel('Survival Probability')
plt.legend(title='Group')
plt.show()

In [ ]:
for group in clean['cat2'].unique():
    group_data = clean[clean['cat2'] == group]
    kmf.fit(group_data['life'], event_observed=group_data['target_label'], label=str(group))
    kmf.plot_survival_function()

plt.title('Survival Curves by Group(cat2)')
plt.xlabel('Time(month)')
plt.ylabel('Survival Probability')
plt.legend(title='Group')
plt.show()

In [ ]:
for group in clean['cat3'].unique():
    group_data = clean[clean['cat3'] == group]
    kmf.fit(group_data['life'], event_observed=group_data['target_label'], label=str(group))
    kmf.plot_survival_function()

plt.title('Survival Curves by Group(cat3)')
plt.xlabel('Time(month)')
plt.ylabel('Survival Probability')
plt.legend(title='Group')
plt.show()

# Statistical Comparison of three categories survival curves using Log-Rank Test

In [ ]:
# Get the unique groups in cat1
groups_cat1 = clean['cat1'].unique()

# Pairwise log-rank tests for all group combinations in cat1
for i in range(len(groups_cat1)):
    for j in range(i + 1, len(groups_cat1)):
        group1 = clean[clean['cat1'] == groups_cat1[i]]
        group2 = clean[clean['cat1'] == groups_cat1[j]]
        
        # Perform log-rank test
        result = logrank_test(
            group1['life'], group2['life'],
            event_observed_A=group1['target_label'],
            event_observed_B=group2['target_label']
        )
        
        print(f"Log-rank test between {groups_cat1[i]} and {groups_cat1[j]}:")
        print(f"P-value: {result.p_value}\n")


Significant Comparisons (p < 0.05):
- D vs A (p = 2.6e-49): Strong evidence of a difference.
- D vs B (p = 0.0349): Moderate evidence of a difference.
- D vs C (p = 0.0078): Strong evidence of a difference.
- A vs B (p = 1.6e-61): Extremely strong evidence of a difference.
- A vs C (p = 1.85e-07): Strong evidence of a difference.
- B vs C (p = 0.00023): Strong evidence of a difference.

Non-Significant Comparisons (p ≥ 0.05):
- D vs N/A (p = 0.5006): No evidence of a difference.
- A vs N/A (p = 0.7213): No evidence of a difference.
- B vs N/A (p = 0.4765): No evidence of a difference.
- C vs N/A (p = 0.5690): No evidence of a difference.

In [ ]:
groups_cat2 = clean['cat2'].unique()

for i in range(len(groups_cat2)):
    for j in range(i + 1, len(groups_cat2)):
        group1 = clean[clean['cat2'] == groups_cat2[i]]
        group2 = clean[clean['cat2'] == groups_cat2[j]]
        
        result = logrank_test(
            group1['life'], group2['life'],
            event_observed_A=group1['target_label'],
            event_observed_B=group2['target_label']
        )
        
        print(f"Log-rank test between {groups_cat2[i]} and {groups_cat2[j]}:")
        print(f"P-value: {result.p_value}\n")


Significant Comparisons (p < 0.05):
- B vs D (p = 0.0): Indicates a very strong difference in survival patterns.
- B vs C (p = 1.88e-51): Extremely strong evidence of a difference.
- B vs N/A (p = 0.0193): Moderate evidence of a difference.
- B vs A (p = 2.2e-05): Strong evidence of a difference.
- D vs C (p = 1.17e-139): Extremely strong evidence of a difference.
- D vs N/A (p = 6.3e-14): Extremely strong evidence of a difference.
- D vs A (p = 1.28e-240): Extremely strong evidence of a difference.
- C vs A (p = 1.25e-18): Strong evidence of a difference.

Non-Significant Comparisons (p ≥ 0.05):
- C vs N/A (p = 0.296): No evidence of a difference.
- N/A vs A (p = 0.414): No evidence of a difference.

In [100]:
groups_cat3 = clean['cat3'].unique()

for i in range(len(groups_cat3)):
    for j in range(i + 1, len(groups_cat3)):
        group1 = clean[clean['cat3'] == groups_cat3[i]]
        group2 = clean[clean['cat3'] == groups_cat3[j]]
        
        result = logrank_test(
            group1['life'], group2['life'],
            event_observed_A=group1['target_label'],
            event_observed_B=group2['target_label']
        )
        
        # print(f"Log-rank test between {groups_cat3[i]} and {groups_cat3[j]}:")
        # print(f"P-value: {result.p_value}\n")


# Feature Engineering

uses cox proportional hazards model

## Removing inconsistent id's

hardcoded removing id's where contract_end_date was incorrectly inputted and later "revised" but became a new data entry in 2020. paige will create a solution later.

In [ ]:
# Prepare the predictor variables
cleaning_X = clean.drop(columns=['join_date', 'contract_end_date', 'target_label', 'target_date', 'life', 
                        'cat1', 'cat2', 'cat3', 'x2_cumulative', 'x1_cumulative', 'remain_contract_month', 'year', 'month', 'day','Category_Combo2'])

cleaning_X.head()


In [ ]:
null_ids = cleaning_X[cleaning_X['last_avg_contract_ratio'].isna()]['id']
null_ids

In [103]:
clean = clean[~clean['id'].isin(null_ids)]

## Choosing important features

In [ ]:
# Prepare survival target
y = Surv.from_dataframe('target_label', 'life', clean)

# Prepare the predictor variables
X = clean.drop(columns=['id', 'join_date', 'contract_end_date', 'target_label', 'target_date', 'life', 
                        'cat1', 'cat2', 'cat3', 'x2_cumulative', 'x1_cumulative', 'remain_contract_month', 'year', 'month', 'day','Category_Combo2'])


# Train-test split (80% train, 20% test) - stratify using the event column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=clean['target_label'])

# Train the Cox Proportional Hazards Model
cox_model = CoxPHSurvivalAnalysis(tol=1e-6, alpha=0.1)  # Using penalization to prevent overfitting
cox_model.fit(X_train, y_train)

# Evaluate on test set using concordance index
c_index_test = concordance_index_censored(y_test['target_label'], y_test['life'], cox_model.predict(X_test))[0]
print(f"Test C-index: {c_index_test:.4f}")

cv = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = []

for train_idx, val_idx in cv.split(X_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]  # Directly indexing `y_train`
    
    # Fit model on training data
    cox_model.fit(X_tr, y_tr)
    
    # Predict on validation data and compute C-index
    c_index = concordance_index_censored(y_val['target_label'], y_val['life'], cox_model.predict(X_val))[0]
    cv_scores.append(c_index)

print(f"Cross-validation C-index scores: {cv_scores}")
print(f"Mean C-index (cross-validation): {np.mean(cv_scores):.4f}")

### Importance

In [ ]:
# Extract Feature Importance (Coefficients)
importance_df = pd.DataFrame({
    'covariate': X.columns,
    'Importance': cox_model.coef_
}).sort_values(by='Importance', ascending=False)

# Display feature importance
print(importance_df)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(importance_df['covariate'], importance_df['Importance'])
plt.xlabel('Feature Importance (Coefficients)')
plt.title('Feature Importance from Cox Proportional Hazards Model')
plt.show()

### MultiCollinearity

In [ ]:
vif_data = pd.DataFrame()
vif_data["covariate"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data

### P-values

In [ ]:
# Convert data to lifelines format
lifelines_df = clean.drop(columns=['id', 'join_date', 'contract_end_date', 'target_date', 'cat1', 'cat2', 'cat3','remain_contract_month', 'year', 'month', 'day'])
lifelines_df['event'] = lifelines_df['target_label']
lifelines_df['time'] = lifelines_df['life']

# Fit Cox model using lifelines
cph = CoxPHFitter()
cph.fit(lifelines_df, duration_col='time', event_col='event')

# Print summary (includes coefficients, standard errors, and p-values)
print(cph.summary)

#### Significant features using P-value

In [ ]:
# Fit Cox model using lifelines
cph = CoxPHFitter()
cph.fit(lifelines_df, duration_col='time', event_col='event')

# Print summary (includes coefficients, standard errors, and p-values)
summary_df = cph.summary

# Filter for features with p-value < 0.05 (significant)
feature_significance = summary_df[['p']]  # Keep all needed columns
feature_significance

### Final set of important features

In [ ]:
important_features = feature_significance
important_features = important_features.merge(vif_data, on='covariate')
important_features = important_features.merge(importance_df, on='covariate')
important_features['Importance'] = abs(important_features['Importance'])

In [ ]:
important_features = important_features[important_features['p'] < 0.05]
important_features.sort_values(by='VIF')
important_features = important_features[important_features['Importance'] > 0.05]
important_features

## Model using CHOSEN Important Features

In [ ]:
# Prepare the survival target (ensure correct column names)
y = Surv.from_dataframe('target_label', 'life', clean)

# Prepare the predictor variables
X = clean[['last_x1', 'x1/x2', 'x2_cumulative_min', 'last_avg_contract_ratio', 'longest_contract', 'total_contract_count']]

# Train-test split (80% train, 20% test) - stratify using the event column
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=clean['target_label'])

# Train the Cox Proportional Hazards Model
cox_model = CoxPHSurvivalAnalysis(tol=1e-6, alpha=0.1)  # Using penalization to prevent overfitting
cox_model.fit(X_train, y_train)

# Evaluate on test set using concordance index
c_index_test = concordance_index_censored(y_test['target_label'], y_test['life'], cox_model.predict(X_test))[0]
print(f"Test C-index: {c_index_test:.4f}")

cv = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = []

for train_idx, val_idx in cv.split(X_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]  # Directly indexing `y_train`
    
    # Fit model on training data
    cox_model.fit(X_tr, y_tr)
    
    # Predict on validation data and compute C-index
    c_index = concordance_index_censored(y_val['target_label'], y_val['life'], cox_model.predict(X_val))[0]
    cv_scores.append(c_index)

print(f"Cross-validation C-index scores: {cv_scores}")
print(f"Mean C-index (cross-validation): {np.mean(cv_scores):.4f}")

In [ ]:
# Extract Feature Importance (Coefficients)
importance_df = pd.DataFrame({
    'covariate': X.columns,
    'Importance': cox_model.coef_
}).sort_values(by='Importance', ascending=False)

# Display feature importance
print(importance_df)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(importance_df['covariate'], importance_df['Importance'])
plt.xlabel('Feature Importance (Coefficients)')
plt.title('Feature Importance from Cox Proportional Hazards Model')
plt.show()

In [ ]:
# Convert data to lifelines format
lifelines_df = clean[['target_label', 'life', 'last_x1', 'x1/x2', 'x2_cumulative_min', 'last_avg_contract_ratio', 'total_contract_count','longest_contract']]
lifelines_df['event'] = lifelines_df['target_label']
lifelines_df['time'] = lifelines_df['life']

# Fit Cox model using lifelines
cph = CoxPHFitter()
cph.fit(lifelines_df, duration_col='time', event_col='event')

# Print summary (includes coefficients, standard errors, and p-values)
print(cph.summary)

In [ ]:
# Fit Cox model using lifelines
cph = CoxPHFitter()
cph.fit(lifelines_df, duration_col='time', event_col='event')

# Print summary (includes coefficients, standard errors, and p-values)
summary_df = cph.summary

# Filter for features with p-value < 0.05 (significant)
feature_significance = summary_df[['p']]  # Keep all needed columns
feature_significance

In [ ]:
vif_data = pd.DataFrame()
vif_data["covariate"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data

### Chosen Important Features

In [ ]:
important_features_chosen = feature_significance
important_features_chosen = important_features_chosen.merge(vif_data, on='covariate')
important_features_chosen = important_features_chosen.merge(importance_df, on='covariate')
important_features_chosen['Importance_Interpret'] = important_features_chosen['Importance']
important_features_chosen['Importance'] = abs(important_features_chosen['Importance'])

important_features_chosen.sort_values('Importance_Interpret', ascending=False)
important_features_chosen

In [ ]:
important_features_chosen['covariate'].tolist()

# COX PROPORTIONAL MODEL

In [ ]:
important_features = [
    'last_x1', 'x1/x2', 'x2_cumulative_min', 
    'avg_contract_length', 'longest_contract', 'total_contract_count']

# Prepare the survival target
y = Surv.from_dataframe("target_label", "life", clean)

# Prepare the predictor variables using only the important features
X = clean[important_features]

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=clean["target_label"]
)

# Train the Cox Proportional Hazards Model
cox_model = CoxPHSurvivalAnalysis(tol=1e-6, alpha=0.1)  # L2 Regularization to prevent overfitting
cox_model.fit(X_train, y_train)

In [ ]:
# Extract 'life' (event/censoring time) from y_test
life_values = y_test["life"]  

# Get the maximum observed time from training data
max_time = y_train["life"].max()

# Create a dictionary to store predictions
future_predictions = {"life + 12": [], "life + 18": [], "life + 24": [], "life + 36": []}

# Get survival functions for each customer
survival_functions = cox_model.predict_survival_function(X_test)

# Define future time points in absolute months (not adding to customer age)
time_offsets = [12, 18, 24, 36]  # Months

# Iterate over each individual's survival function and life time
for fn, life in zip(survival_functions, y_test["life"]):
    
    # Predict survival probability at life + t (but cap at max_time)
    for t in time_offsets:
        adjusted_time = min(life + t, max_time)  # Ensure we don't exceed max_time
        future_predictions[f"life + {t}"].append(fn(adjusted_time))  # Use adjusted time


# Convert predictions to DataFrame
future_pred_df = pd.DataFrame(future_predictions, index=X_test.index)

# Retrieve customer IDs from the original dataset
customer_ids = clean.loc[X_test.index, "id"]

# Add Customer IDs to the predictions DataFrame
future_pred_df["id"] = customer_ids

# Move 'Customer ID' to the first column for better readability
future_pred_df = future_pred_df.set_index("id")

# Display first few rows
print(future_pred_df.head(10))




## integrated brier score for cox proportional model

In [ ]:

# Convert y_train and y_test into structured arrays for sksurv compatibility
survival_train = np.array([(event, time) for event, time in zip(y_train["target_label"], y_train["life"])],
                          dtype=[('event', bool), ('time', float)])

survival_test = np.array([(event, time) for event, time in zip(y_test["target_label"], y_test["life"])],
                         dtype=[('event', bool), ('time', float)])

# Define time points at which to evaluate the Brier score
times = np.percentile(y_test["life"], np.linspace(10, 90, 20))  # Select meaningful time points

# Predict survival functions for test data
survival_preds = cox_model.predict_survival_function(X_test)

# Convert predictions into an estimate matrix
estimate = np.vstack([fn(times) for fn in survival_preds])

# Compute Brier scores at different time points
brier_scores = brier_score(survival_train, survival_test, estimate, times)

# Extract the Brier scores (second element in the tuple)
brier_scores = brier_scores[1]

# Now compute the Integrated Brier Score
integrated_brier_score = trapezoid(brier_scores, times) / (times.max() - times.min())

# Convert to scalar
integrated_brier_score = float(integrated_brier_score)

print(f"Integrated Brier Score for Cox Model: {integrated_brier_score:.4f}")


In [ ]:

# Convert y_train and y_test into structured arrays for sksurv compatibility
survival_train = np.array([(event, time) for event, time in zip(y_train["target_label"], y_train["life"])],
                          dtype=[('event', bool), ('time', float)])

survival_test = np.array([(event, time) for event, time in zip(y_test["target_label"], y_test["life"])],
                         dtype=[('event', bool), ('time', float)])

# Define time points at which to evaluate the Brier score
times = np.percentile(y_test["life"], np.linspace(20, 90, 10))  # Select meaningful time points

# Predict survival functions for test data
survival_preds = cox_model.predict_survival_function(X_test)

# Convert predictions into an estimate matrix
estimate = np.row_stack([fn(times) for fn in survival_preds])

# Compute Brier scores at different time points
brier_scores = brier_score(survival_train, survival_test, estimate, times)

# Compute Integrated Brier Score (mean of all time points)
integrated_brier_score = np.mean(brier_scores)

# Display the Integrated Brier Score
print(f"Integrated Brier Score for Cox Model: {integrated_brier_score:.4f}")


In [ ]:
future_pred_df.sort_values(by="life + 36", ascending=True)

In [ ]:
sorted_future_cox_36_month = future_pred_df["life + 36"].sort_values(ascending=True)
sorted_future_cox_36_month

In [ ]:
sorted_future_cox_12_month = future_pred_df["life + 12"].sort_values(ascending=True)
sorted_future_cox_12_month

In [ ]:
sorted_future_cox_18_month = future_pred_df["life + 18"].sort_values(ascending=True)
sorted_future_cox_18_month

# WEIBULL - uses weibull distribution for future predictions
### this is what we presented in the mid point

In [124]:
# Define predictor features
important_features = [
 'last_x1', 'x1/x2', 'x2_cumulative_min',
 'longest_contract', 'last_avg_contract_ratio', 'total_contract_count']

# Prepare survival target (event indicator and survival time)
y = clean[["life", "target_label"]]  # Target: survival time & event indicator
X = clean[important_features]  # Predictor variables

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y["target_label"]
)


In [ ]:
# Fit Weibull AFT Model
aft_model = WeibullAFTFitter()
aft_model.fit(y_train.join(X_train), duration_col="life", event_col="target_label")

# Display model summary
aft_model.print_summary()


In [ ]:
#Filter active customers (target_label != 1)
active_customers = y_test[y_test["target_label"] != 1]

#Extract 'age' of each customer
customer_ages = active_customers["life"]  # Ensure "age" exists in dataset

#Keep corresponding features
X_active = X_test.loc[active_customers.index]

#Define prediction time offsets
time_offsets = [12, 24, 36, 48]

#Dictionary to store predictions
future_aft_predictions = {}

#Iterate over time offsets and predict for each customer's future age
for t in time_offsets:
    future_times = customer_ages + t  # Compute future time for each customer

    #Predict survival function at these specific times for each customer
    survival_probs = np.array([
        aft_model.predict_survival_function(X_active.iloc[i:i+1], times=[future_times.iloc[i]]).values.flatten()[0]
        for i in range(len(X_active))
    ])

    #Store survival probability for each customer at their specific future time
    future_aft_predictions[f"life + {t}"] = survival_probs

#Convert predictions to DataFrame
future_aft_df = pd.DataFrame(future_aft_predictions, index=X_active.index)

# Add customer IDs for reference
customer_ids = clean.loc[X_active.index, "id"]
future_aft_df["id"] = customer_ids

#Move ID column to front
future_aft_df = future_aft_df.set_index("id")

#Display results
future_aft_df

In [ ]:
specific_id = "f7194e770adcb3c8e92c8cb79cfc0615"
row = future_aft_df.loc[specific_id]
print(row)

In [ ]:
# Sort DataFrame in ascending order based on 'life + 12' survival probability
sorted_aft_df = future_aft_df.sort_values(by="life + 12", ascending=True)

# Display first few rows
print(sorted_aft_df.head(10))


## Brier score at A months predicting B months after(Weibull)

In [129]:
train_idx, test_idx = train_test_split(
    clean.index, test_size=0.2, random_state=42, stratify=y["target_label"]
)

# Retrieve the 'id' values for the test set
id_test = clean.loc[test_idx, "id"]

In [ ]:
# Time point = A months, predicting B months

A = 156
B = 168

score_df = create_df(df, A, B, id_test)

# Prepare the survival target
score_df_y = Surv.from_dataframe("target_label", "time_step", score_df)

# Prepare the predictor variables using only the important features
score_df_X = score_df[important_features]

# Assume df contains 'time' (event/censoring time) and 'target_label' (churn status)
time_t = B  # Specify the time at which to compute the Brier Score

# Predict survival function for each customer
surv_funcs = aft_model.predict_survival_function(score_df_X)

# Extract survival probability at time_t
surv_probs = surv_funcs.loc[time_t].values

# Compute Brier Score
y_true = 1 - score_df["target_label"]  # Flip 1 → 0 (churned), 0 → 1 (alive)
brier = np.mean((surv_probs - y_true) ** 2)

print(f"Brier Score at t={time_t}: {brier}")

## Brier, Recall, Precision, Specificity, and Accuracy at A months predicting 12 months after(Weibull)

In [ ]:
# Initialize an empty list to store results
results = []

# Loop over A values from 12 to 168 in steps of 12
for A in range(12, 157, 12):  
    B = A + 12  # Define B as A + 12

    score_df = create_df(df, A, B, id_test)

    # Prepare the survival target
    score_df_y = Surv.from_dataframe("target_label", "time_step", score_df)

    # Prepare the predictor variables using only the important features
    score_df_X = score_df[important_features]

    # Assume df contains 'time' (event/censoring time) and 'target_label' (churn status)
    time_t = B  # Specify the time at which to compute the Brier Score

    # Predict survival function for each customer
    surv_funcs = aft_model.predict_survival_function(score_df_X)

    # Extract survival probability at time_t
    surv_probs = surv_funcs.loc[time_t].values

    # Compute Brier Score
    y_true = 1 - score_df["target_label"]  # Flip 1 → 0 (churned), 0 → 1 (alive)
    brier = np.mean((surv_probs - y_true) ** 2)

        # Convert survival probabilities into binary predictions
    y_pred = (surv_probs < 0.6).astype(int)  # Churn if survival probability < 0.5
    y_true_original = score_df["target_label"].values  # Use original labels

    # Compute confusion matrix
    conf_matrix = confusion_matrix(y_true_original, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()

    # Compute classification metrics
    accuracy = accuracy_score(y_true_original, y_pred)
    precision = precision_score(y_true_original, y_pred, zero_division=0)
    recall = recall_score(y_true_original, y_pred)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # Avoid division by zero

    # Store the result
    results.append({
        "Model": "Weibull",
        "Time point": A,
        "Future time": B,
        "Brier Score": round(brier, 3),
        "Accuracy": round(accuracy, 3),
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "Specificity": round(specificity, 3)
    })

# Convert results to DataFrame
brier_df_12 = pd.DataFrame(results)
print(brier_df_12)

In [ ]:
brier_df_12['Brier Score'].mean()

## Brier, Recall, Precision, Specificity, and Accuracy at A months predicting 24 months after(Weibull)

In [ ]:
# Initialize an empty list to store results
results = []

# Loop over A values from 12 to 168 in steps of 12
for A in range(12, 157, 12):  
    B = A + 24  # Define B as A + 24

    score_df = create_df(df, A, B, id_test)

    # Prepare the survival target
    score_df_y = Surv.from_dataframe("target_label", "time_step", score_df)

    # Prepare the predictor variables using only the important features
    score_df_X = score_df[important_features]

    # Assume df contains 'time' (event/censoring time) and 'target_label' (churn status)
    time_t = B  # Specify the time at which to compute the Brier Score

    # Predict survival function for each customer
    surv_funcs = aft_model.predict_survival_function(score_df_X)

    # Extract survival probability at time_t
    surv_probs = surv_funcs.loc[time_t].values

    # Compute Brier Score
    y_true = 1 - score_df["target_label"]  # Flip 1 → 0 (churned), 0 → 1 (alive)
    brier = np.mean((surv_probs - y_true) ** 2)

        # Convert survival probabilities into binary predictions
    y_pred = (surv_probs < 0.6).astype(int)  # Churn if survival probability < 0.5
    y_true_original = score_df["target_label"].values  # Use original labels

    # Compute confusion matrix
    conf_matrix = confusion_matrix(y_true_original, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()

    # Compute classification metrics
    accuracy = accuracy_score(y_true_original, y_pred)
    precision = precision_score(y_true_original, y_pred, zero_division=0)
    recall = recall_score(y_true_original, y_pred)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # Avoid division by zero

    # Store the result
    results.append({
        "Model": "Weibull",
        "Time point": A,
        "Future time": B,
        "Brier Score": round(brier, 3),
        "Accuracy": round(accuracy, 3),
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "Specificity": round(specificity, 3)
    })

# Convert results to DataFrame
brier_df_24 = pd.DataFrame(results)
print(brier_df_24)

In [ ]:
brier_df_24['Brier Score'].mean()

## Brier, Recall, Precision, Specificity, and Accuracy at A months predicting 36 months after(Weibull)

In [ ]:
# Initialize an empty list to store results
results = []

# Loop over A values from 12 to 168 in steps of 12
for A in range(12, 157, 12):  
    B = A + 36  # Define B as A + 36

    score_df = create_df(df, A, B, id_test)

    # Prepare the survival target
    score_df_y = Surv.from_dataframe("target_label", "time_step", score_df)

    # Prepare the predictor variables using only the important features
    score_df_X = score_df[important_features]

    # Assume df contains 'time' (event/censoring time) and 'target_label' (churn status)
    time_t = B  # Specify the time at which to compute the Brier Score

    # Predict survival function for each customer
    surv_funcs = aft_model.predict_survival_function(score_df_X)

    # Extract survival probability at time_t
    surv_probs = surv_funcs.loc[time_t].values

    # Compute Brier Score
    y_true = 1 - score_df["target_label"]  # Flip 1 → 0 (churned), 0 → 1 (alive)
    brier = np.mean((surv_probs - y_true) ** 2)

        # Convert survival probabilities into binary predictions
    y_pred = (surv_probs < 0.6).astype(int)  # Churn if survival probability < 0.5
    y_true_original = score_df["target_label"].values  # Use original labels

    # Compute confusion matrix
    conf_matrix = confusion_matrix(y_true_original, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()

    # Compute classification metrics
    accuracy = accuracy_score(y_true_original, y_pred)
    precision = precision_score(y_true_original, y_pred, zero_division=0)
    recall = recall_score(y_true_original, y_pred)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # Avoid division by zero

    # Store the result
    results.append({
        "Model": "Weibull",
        "Time point": A,
        "Future time": B,
        "Brier Score": round(brier, 3),
        "Accuracy": round(accuracy, 3),
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "Specificity": round(specificity, 3)
    })

# Convert results to DataFrame
brier_df_36 = pd.DataFrame(results)
print(brier_df_36)

In [ ]:
brier_df_36['Brier Score'].mean()

## Brier score plot

In [ ]:
# Define X-axis break points
x_ticks = sorted(brier_df_12["Time point"].unique())  # Ensure breaks are correct

# Plot each dataframe as a separate line
plt.plot(brier_df_12["Time point"], brier_df_12["Brier Score"], marker='o', label="Predicting 12 months after")
plt.plot(brier_df_24["Time point"], brier_df_24["Brier Score"], marker='s', label="Predicting 24 months after")
plt.plot(brier_df_36["Time point"], brier_df_36["Brier Score"], marker='^', label="Predicting 36 months after")

# Labels and title
plt.xlabel("Time Point")
plt.ylabel("Brier Score")
plt.title("Brier Score at Time Point")
plt.legend()
plt.grid(True)

# Set x-axis breaks at 12, 24, 36, etc.
plt.xticks(x_ticks)

# Show the plot
plt.show()


## Recall score plot

In [ ]:
# Define X-axis break points
x_ticks = sorted(brier_df_12["Time point"].unique())  # Ensure breaks are correct

# Plot each dataframe as a separate line
plt.plot(brier_df_12["Time point"], brier_df_12["Recall"], marker='o', label="Predicting 12 months after")
plt.plot(brier_df_24["Time point"], brier_df_24["Recall"], marker='s', label="Predicting 24 months after")
plt.plot(brier_df_36["Time point"], brier_df_36["Recall"], marker='^', label="Predicting 36 months after")

# Labels and title
plt.xlabel("Time Point")
plt.ylabel("Recall Score")
plt.title("Recall Score at Time Point")
plt.legend()
plt.grid(True)

# Set x-axis breaks at 12, 24, 36, etc.
plt.xticks(x_ticks)

# Show the plot
plt.show()

## Accuracy score plot

In [ ]:
# Define X-axis break points
x_ticks = sorted(brier_df_12["Time point"].unique())  # Ensure breaks are correct

# Plot each dataframe as a separate line
plt.plot(brier_df_12["Time point"], brier_df_12["Accuracy"], marker='o', label="Predicting 12 months after")
plt.plot(brier_df_24["Time point"], brier_df_24["Accuracy"], marker='s', label="Predicting 24 months after")
plt.plot(brier_df_36["Time point"], brier_df_36["Accuracy"], marker='^', label="Predicting 36 months after")

# Labels and title
plt.xlabel("Time Point")
plt.ylabel("Accuracy Score")
plt.title("Accuracy Score at Time Point")
plt.legend()
plt.grid(True)

# Set x-axis breaks at 12, 24, 36, etc.
plt.xticks(x_ticks)

# Show the plot
plt.show()

# CUSTOMER INSTABILITY INDEX

In [140]:
active_customers = clean[clean["target_label"] == 0]

In [ ]:
active_customers["last_x1/x1_mean"] = active_customers["last_x1"] / active_customers["x1_mean"]
active_customers["last_x2/x2_mean"] = active_customers["last_x2"] / active_customers["x2_mean"]

In [142]:
pd.set_option('display.float_format', '{:.6f}'.format)

In [143]:
customer_ages = active_customers["life"]

# Extract corresponding predictor features for the active customers
X_active = active_customers[important_features]  # Use the full feature set for all active customers

# Define prediction time offset for life + 12
time_offset = 12

# Compute future time for each customer
future_times = customer_ages + time_offset

# Predict survival function at these specific times for each customer
survival_probs_12 = np.array([
    aft_model.predict_survival_function(X_active.iloc[i:i+1], times=[future_times.iloc[i]]).values.flatten()[0]
    for i in range(len(X_active))
])

# Convert predictions to DataFrame
future_aft_df_12 = pd.DataFrame({"id": active_customers["id"], "life + 12": survival_probs_12})

# Set ID as index
future_aft_df_12 = future_aft_df_12.set_index("id")

In [144]:
churn_aft_df_12 = 1 - future_aft_df_12

In [146]:
risk_factors = active_customers[["id", "last_x1/x1_mean", "last_x2/x2_mean", "remain_contract_month", "last_avg_contract_ratio"]]

# Reset index for future survival probabilities
#churn_probs_12 = churn_aft_df_12.reset_index()
future_probs_12 = future_aft_df_12.reset_index()

# Merge survival probabilities with risk factors
risk_df_12 = future_probs_12.merge(risk_factors, on="id", how="outer")


# Normalize each feature to be within 0-1
risk_df_12["last_avg_contract_ratio"] = risk_df_12["last_avg_contract_ratio"] 
risk_df_12["last_x1/x1_mean"] = risk_df_12["last_x1/x1_mean"] / risk_df_12["last_x1/x1_mean"].max()
risk_df_12["last_x2/x2_mean"] = risk_df_12["last_x2/x2_mean"] / risk_df_12["last_x2/x2_mean"].max()
risk_df_12["remain_contract_month"] = risk_df_12["remain_contract_month"] / risk_df_12["remain_contract_month"].max()

# Compute risk score with normalized features
risk_df_12["risk_score"] = (
    risk_df_12["life + 12"] * 0.70 +  # Survival probability is already between 0-1
    risk_df_12["last_avg_contract_ratio"] * 0.15 +
    risk_df_12["last_x1/x1_mean"] * 0.05 +
    risk_df_12["last_x2/x2_mean"] * 0.05 +
    risk_df_12["remain_contract_month"] * 0.05
) * 100  # Scale to 100

# # Clip values to ensure risk score stays within 0-100
# risk_df_12["risk_score"] = risk_df_12["risk_score"].clip(0, 100)

# Keep only ID and risk score
risk_df_12 = risk_df_12[["id", "risk_score"]]

In [147]:
customer_ages = active_customers["life"]

# Extract corresponding predictor features for the active customers
X_active = active_customers[important_features]  # Use the full feature set for all active customers

# Define prediction time offset for life + 24
time_offset = 24

# Compute future time for each customer
future_times = customer_ages + time_offset

# Predict survival function at these specific times for each customer
survival_probs_24 = np.array([
    aft_model.predict_survival_function(X_active.iloc[i:i+1], times=[future_times.iloc[i]]).values.flatten()[0]
    for i in range(len(X_active))
])

# Convert predictions to DataFrame
future_aft_df_24 = pd.DataFrame({"id": active_customers["id"], "life + 24": survival_probs_24})

# Set ID as index
future_aft_df_24 = future_aft_df_24.set_index("id")

In [148]:
churn_aft_df_24 = 1 - future_aft_df_24

In [149]:
risk_factors = active_customers[["id", "last_x1/x1_mean", "last_x2/x2_mean", "remain_contract_month", "last_avg_contract_ratio"]]

# Reset index for future survival probabilities
# churn_probs_24 = churn_aft_df_24.reset_index()
future_probs_24 = future_aft_df_24.reset_index()

# Merge survival probabilities with risk factors
risk_df_24 = future_probs_24.merge(risk_factors, on="id", how="outer")

# Normalize each feature to be within 0-1
risk_df_24["last_avg_contract_ratio"] = risk_df_24["last_avg_contract_ratio"] 
risk_df_24["last_x1/x1_mean"] = risk_df_24["last_x1/x1_mean"] / risk_df_24["last_x1/x1_mean"].max()
risk_df_24["last_x2/x2_mean"] = risk_df_24["last_x2/x2_mean"] / risk_df_24["last_x2/x2_mean"].max()
risk_df_24["remain_contract_month"] = risk_df_24["remain_contract_month"] / risk_df_24["remain_contract_month"].max()

# Compute risk score with normalized features
risk_df_24["risk_score"] = (
    risk_df_24["life + 24"] * 0.70 +  # Survival probability is already between 0-1
    risk_df_24["last_avg_contract_ratio"] * 0.15 +
    risk_df_24["last_x1/x1_mean"] * 0.05 +
    risk_df_24["last_x2/x2_mean"] * 0.05 +
    risk_df_24["remain_contract_month"] * 0.05
) * 100  # Scale to 100

# Keep only ID and risk score
risk_df_24 = risk_df_24[["id", "risk_score"]]

### combining 12 and 24 month - final approach using the 12 and 24 month weighted average

In [150]:
# churn_probs_12 = churn_aft_df_12.reset_index()
# churn_probs_24 = churn_aft_df_24.reset_index()


# Merge churn probabilities for life + 12 and life + 24 with risk factors
risk_df_12_24 = future_probs_12.merge(future_probs_24, on="id", how="outer").merge(risk_factors, on="id", how="outer")

# Normalize each feature to be within 0-1
risk_df_12_24["last_avg_contract_ratio"] = risk_df_12_24["last_avg_contract_ratio"]
risk_df_12_24["last_x1/x1_mean"] = risk_df_12_24["last_x1/x1_mean"] / risk_df_12_24["last_x1/x1_mean"].max()
risk_df_12_24["last_x2/x2_mean"] = risk_df_12_24["last_x2/x2_mean"] / risk_df_12_24["last_x2/x2_mean"].max()
risk_df_12_24["remain_contract_month"] = risk_df_12_24["remain_contract_month"] / risk_df_12_24["remain_contract_month"].max()

# Compute updated risk score with new weightage
risk_df_12_24["risk_score"] = (
    risk_df_12_24["life + 12"] * 0.40 +  # Weight 40% for life + 12
    risk_df_12_24["life + 24"] * 0.30 +  # Weight 30% for life + 24
    risk_df_12_24["last_avg_contract_ratio"] * 0.15 +
    risk_df_12_24["last_x1/x1_mean"] * 0.05 +
    risk_df_12_24["last_x2/x2_mean"] * 0.05 +
    risk_df_12_24["remain_contract_month"] * 0.05
) * 1  # No additional scaling

# Keep only ID and risk score
risk_df_12_24 = risk_df_12_24[["id", "risk_score"]]

# Sort by highest risk score
risk_df_12_24_sorted = risk_df_12_24.sort_values(by="risk_score", ascending=False)

In [151]:
risk_df_12_24["risk_score"] = pd.to_numeric(risk_df_12_24["risk_score"], errors="coerce")

# Create a new dataframe with 1 - risk_score
risk_df_inverse = risk_df_12_24.copy()
risk_df_inverse["inverse_risk_score"] = 1 - risk_df_inverse["risk_score"]

# Keep only ID and inverse risk score
risk_df_inverse = risk_df_inverse[["id", "inverse_risk_score"]]

In [ ]:
risk_df_inverse.sort_values(by="inverse_risk_score", ascending=False)

### CII per category combo

In [153]:
clean['Category_Combo'] = clean['cat1'] + '-' + clean['cat2']

In [154]:
risk_merged = risk_df_inverse.merge(clean[['id', 'Category_Combo']], on='id', how='left')

In [155]:
risk_merged.to_csv('risk_merged.csv', index=False)

In [ ]:
risk_merged